In [185]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

%matplotlib inline


import pandas as pd
import numpy as np
import scipy as sp
import patsy
import pystan
#from pystan import StanFit4model

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()



R = ro.r

import statsmodels.api as sm

import rpy2.robjects as ro
#import pandas.rpy.common as com

import matplotlib.pyplot as platform
pd.set_option('max_columns', 50)

from decimal import *

print getcontext()




Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999999, Emax=999999999, capitals=1, flags=[], traps=[Overflow, DivisionByZero, InvalidOperation])


In [177]:

url = "http://stat.columbia.edu/~gelman/arm/examples/arsenic/wells.dat"
wells = pd.read_table(url, sep='\s+', header=0, index_col=0)
wells["dist100"] = wells['dist']/100

y = wells['switch']
wellsNp = wells.as_matrix()
y_mod = wellsNp[:, 0]

y_hat = y.transpose()

y_mean = np.mean(y_mod)

s = sp.special.logit(y_mean)

logit = np.log(y_mean / (1 - y_mean))

f = "~ 0 + dist + arsenic"
mat = patsy.dmatrix(f, wells, return_type='dataframe')
nrow, ncol = mat.shape


data_loo = {'N': nrow , 'P': ncol, 'y': y, 'x': mat, 'a': s }


   switch  arsenic       dist  assoc  educ
1       1     2.36  16.826000      0     0
2       1     0.71  47.321999      0     0
3       0     2.07  20.966999      0    10
4       1     1.15  21.486000      0    12
5       1     1.10  40.874001      1    14
[ 1.  1.  0. ...,  0.  0.  1.]


In [178]:

logistic = '''
data {
  int<lower=0> N; 
  int<lower=0> P;
  int<lower=0,upper=1> y[N];
  matrix[N,P] x; 
  real a;
}
parameters {
  real beta0;
  vector[P] beta;
}
model {
  beta0 ~ student_t(7, a, 0.1);
  beta ~ student_t(7, 0, 1);
  y ~ bernoulli_logit(beta0 + x * beta);
}
generated quantities {
  vector[N] log_lik;
  for (n in 1:N)
    log_lik[n] <- bernoulli_logit_log(y[n], beta0 + x[n] * beta);
}
'''

In [176]:
fit1 = pystan.stan(model_code=logistic, data=data_loo, iter=1000, chains=4)

print fit1

In [191]:
#log_like = pystan.stanfit4model
#print fit1

parameter_name = "log_lik"
type(fit1)
matrix = fit1.extract(pars=parameter_name)
print matrix['log_lik']
#np.take(fit1, parameter_name)

[[-0.30339556 -0.7058634  -1.21496156 ..., -0.84882572 -0.79490934
  -0.58753698]
 [-0.33984576 -0.68792866 -1.14311882 ..., -0.86652432 -0.81022652
  -0.57661097]
 [-0.36504355 -0.74197482 -1.08272063 ..., -0.77753159 -0.73659556
  -0.63964319]
 ..., 
 [-0.35989739 -0.75971277 -1.08815556 ..., -0.75928492 -0.71986293
  -0.65466318]
 [-0.31933174 -0.70103874 -1.18130564 ..., -0.84327952 -0.79273562
  -0.59013805]
 [-0.33255534 -0.76323267 -1.14049535 ..., -0.78802525 -0.73643974
  -0.63754257]]
